In [1]:
from bs4 import BeautifulSoup as bs
import re
import requests
import pandas as pd
import time
import collections
import random

In [ ]:
data_all = pd.read_csv('./1090531_test_try_catch.csv') #2004~2020每場比賽
del data_all['Unnamed: 0'] 
#刪掉三隊
data_all = data_all[(~data_all['主隊'].isin(['中信','誠泰','米迪亞'])) & (~data_all['客隊'].isin(['中信','誠泰','米迪亞']))]
#刪掉投手與非先發
data_all = data_all[(data_all['守備位置'] != 'P')&(~data_all['守備位置'].str.startswith(' ('))]

# 爬蟲

In [ ]:
#舊版打者習慣
Lamigo = data_all[(data_all['隊伍'].isin(['Lamigo','樂天']))] #做lamigo
ID_lst = list(set(Lamigo['球員ID'].values))
print(len(set(Lamigo['球員ID'].values)))
type_ = 1 ######################打者習慣#################
df_total = pd.DataFrame()
num = 0
keep_craw = []
player_url = []
for ID in ID_lst:
    print('現在到：'+ str(num))
    #url = 'http://www.cpbl.com.tw/players/apart.html?player_id={}&teamno='.format(ID)
    url = 'http://www.cpbl.com.tw/players/apart.html?player_id={}'.format(ID)
    try:
        re = requests.get(url) 
        time.sleep(random.uniform(1,2))
        soup = bs(re.text, 'html.parser')
        Year_lst = [option.text for option in soup.find_all("select",{"class":"date_select"})[1].find_all("option")]
        year_lst = [i for i in Year_lst if int(i) >= 2004]
        if len(year_lst) > 0:
            for year in year_lst:
                try:
                    response = requests.get(url + f'&year={year}'+f'&type=0{type_}')
                    time.sleep(random.uniform(1,2))
                    print(url + f'&year={year}'+f'&type=0{type_}')
                    if response.status_code != 200:
                        print('URL發生錯誤：' + url)
                    soup = bs(response.text, 'html.parser')
                    table = soup.find_all('tr')
                    if len(table)>3:
                        #變數名稱
                        title_list = table[2].text.split('\n')
                        title_list[0:2] = ['num','R/L']
                        df = pd.DataFrame(columns = title_list)
                        for i in range(3,len(table)):
                            df.loc[i] = table[i].text.split('\n')
                        df['year'] = year
                        df['ID'] = ID
                        df['team'] = soup.find('div','h1_side').a['href'].replace('/web/team_brief.php?&team=','')
                        df_total = pd.concat([df_total,df])
                except:
                    keep_craw.append(url + f'&year={year}'+f'&type=0{type_}')
                    print('這年開不了')
                    continue
                response.close()
    except:
        print("這個球員開不了")
        player_url.append(url)
    num +=1
print(len(keep_craw))
print(len(player_url))
df_total

In [ ]:
temp = df_total.copy()
temp

In [ ]:
for i in keep_craw:
    response = requests.get(i)
    time.sleep(random.uniform(1,2))
    soup = bs(response.text, 'html.parser')
    table = soup.find_all('tr')
    if len(table)>3:
        #變數名稱
        title_list = table[2].text.split('\n')
        title_list[0:2] = ['num','R/L']
        df = pd.DataFrame(columns = title_list)
        for i in range(3,len(table)):
            df.loc[i] = table[i].text.split('\n')
        df['year'] = year
        df['ID'] = ID
        df['team'] = soup.find('div','h1_side').a['href'].replace('/web/team_brief.php?&team=','')
        df_total = pd.concat([df_total,df])
df_total

In [ ]:
df_total.to_csv('./RL_type1.csv',encoding='utf-8',index=False)

In [ ]:
#爬主客順便存月份網址
Lamigo = data_all[(data_all['隊伍'].isin(['Lamigo','樂天']))] #做lamigo
ID_lst = list(set(Lamigo['球員ID'].values))
print(len(set(Lamigo['球員ID'].values)))
type_ = 2 ######################主客#################
df_total = pd.DataFrame()
num = 0
keep_craw = []
player_url = []
month_url = []
for ID in ID_lst:
    print('現在到：'+ str(num))
    #url = 'http://www.cpbl.com.tw/players/apart.html?player_id={}&teamno='.format(ID)
    url = 'http://www.cpbl.com.tw/players/apart.html?player_id={}'.format(ID)
    try:
        re = requests.get(url) 
        time.sleep(random.uniform(1,2))
        soup = bs(re.text, 'html.parser')
        Year_lst = [option.text for option in soup.find_all("select",{"class":"date_select"})[1].find_all("option")]
        year_lst = [i for i in Year_lst if int(i) >= 2004]
        if len(year_lst) > 0:
            for year in year_lst:
                try:
                    month_url.append(url + f'&year={year}'+'&type=05')
                    response = requests.get(url + f'&year={year}'+f'&type=0{type_}')
                    time.sleep(random.uniform(1,2))
                    #print(url + f'&year={year}'+f'&type=0{type_}')
                    if response.status_code != 200:
                        print(print('URL發生錯誤：' + url))
                    soup = bs(response.text, 'html.parser')
                    table = soup.find_all('tr')
                    if len(table)>3:
                        #變數名稱
                        title_list = table[2].text.split('\n')
                        title_list[0:2] = ['num','H/A']
                        df = pd.DataFrame(columns = title_list)
                        for i in range(3,len(table)):
                            df.loc[i] = table[i].text.split('\n')
                        df['year'] = year
                        df['ID'] = ID
                        df['team'] = soup.find('div','h1_side').a['href'].replace('/web/team_brief.php?&team=','')
                        df_total = pd.concat([df_total,df])
                except:
                    keep_craw.append(url + f'&year={year}'+f'&type=0{type_}')
                    print('這年開不了')
                    continue
                response.close()
    except:
        print("這個球員開不了")
        player_url.append(url)
    num +=1
print(len(keep_craw))
print(len(player_url))
print(len(month_url))
df_total

In [ ]:
#temp = df_total.copy()
temp

In [ ]:
for i in keep_craw:
    response = requests.get(i)
    time.sleep(random.uniform(1,2))
    soup = bs(response.text, 'html.parser')
    table = soup.find_all('tr')
    if len(table)>3:
        #變數名稱
        title_list = table[2].text.split('\n')
        title_list[0:2] = ['num','H/A']
        df = pd.DataFrame(columns = title_list)
        for i in range(3,len(table)):
            df.loc[i] = table[i].text.split('\n')
        df['year'] = year
        df['ID'] = ID
        df['team'] = soup.find('div','h1_side').a['href'].replace('/web/team_brief.php?&team=','')
        df_total = pd.concat([df_total,df])
df_total

In [ ]:
temp.to_csv('./HA_867_type2.csv',encoding='utf-8',index=False)

In [ ]:
df_total
df_total.to_csv('./HA_004634_G073.csv',encoding='utf-8',index=False)

In [ ]:
df_total.to_csv('./HA_type2.csv',encoding='utf-8',index=False)
df_total.to_excel('./HA_type2.xlsx',index=False)

In [ ]:
#月份
df_total = pd.DataFrame()
num = 0
keep_craw = []
for url in month_url:
    print('現在到：'+ str(num))
    try:
        response = requests.get(url)
       # print(url)
        time.sleep(random.uniform(1,2))
        if response.status_code != 200:
            print(print('URL發生錯誤：' + url))
        soup = bs(response.text, 'html.parser')
        table = soup.find_all('tr')
        if len(table)>3:
            #變數名稱
            title_list = table[2].text.split('\n')
            title_list[0:2] = ['num','month']
            df = pd.DataFrame(columns = title_list)
            for i in range(3,len(table)):
                df.loc[i] = table[i].text.split('\n')
            df['year'] = url.replace('http://www.cpbl.com.tw/players/apart.html?player_id=','').split('&')[1][-4:]
            df['ID'] = url.replace('http://www.cpbl.com.tw/players/apart.html?player_id=','').split('&')[0]
            df['team'] = soup.find('div','h1_side').a['href'].replace('/web/team_brief.php?&team=','')
            df_total = pd.concat([df_total,df])
    except:
        keep_craw.append(url)
        print('這年開不了')
    num +=1
print(len(keep_craw))
df_total

In [ ]:
df_total

In [ ]:
df_total.to_csv('./month_type5.csv',encoding='utf-8',index=False)